In [8]:
import pandas as pd
import numpy
import re

In [9]:
df = pd.read_csv('NIST_meta.csv')

In [10]:
def fix_location_name(location):
    fixed = location.replace(" ","_")
    if location=="Bath1":
        fixed = "Bathroom_1"
    elif location=="Bath2":
        fixed = "Bathroom_2"
    elif location=="MBath":
        fixed = "Master_Bathroom"
    elif location=="Bedroom2":
        fixed = "Bedroom_2"
    elif location=="Bedroom3":
        fixed = "Bedroom_3"
    elif location=="Bedroom4":
        fixed = "Bedroom_4"
    elif location=="MBedroom":
        fixed = "Master_Bedroom"
    return fixed

In [11]:
df.head(10)

Data Label Subsystem Measurement Location  \
0           DHW_ClothesWasherColdFlow       DHW              Utility   
1            DHW_ClothesWasherHotFlow       DHW              Utility   
2               DHW_DishwasherHotFlow       DHW              Kitchen   
3                 DHW_SHWGlycolTempIn       DHW             Basement   
4                DHW_SHWGlycolTempOut       DHW             Basement   
5  DHW_HeatPumpWaterHeaterEnergyTotal       DHW             Basement   
6   DHW_HeatPumpWaterHeaterPowerTotal       DHW             Basement   
7                DHW_ManifoldColdFlow       DHW             Basement   
8                 DHW_ManifoldHotFlow       DHW             Basement   
9                DHW_MixValveColdFlow       DHW             Basement   

  Measured Parameter                                        Description  \
0         Flow_Water  The cumulative volume of cold water flowing in...   
1         Flow_Water  The cumulative volume of hot water flowing int...   
2         Flow_Water  The cumulative volume of hot water flowing int...   
3        Temp_Glycol  The instantaneous temperature of the glycol-wa...   
4        Temp_Glycol  The instantaneous temperature of the glycol-wa...   
5  Energy_Electrical  Cumulative energy consumption by heat pump wat...   
6   Power_Electrical  Instantaneous power consumption by heat pump w...   
7         Flow_Water  The cumulative volume of water flowing into th...   
8         Flow_Water  The cumulative volume of water flowing into th...   
9         Flow_Water  The cumulative volume of mains water flowing i...   

     Units  
0  Gallons  
1  Gallons  
2  Gallons  
3       °C  
4       °C  
5       Wh  
6        W  
7  Gallons  
8  Gallons  
9  Gallons

In [12]:
header = """@prefix bldg: <https://pages.nist.gov/netzero/data.html#data_dictionary#> .
@prefix brick: <https://brickschema.org/schema/1.1/Brick#> . 
@prefix btag: <https://brickschema.org/schema/1.0.2/BrickTag#> .
@prefix gtc: <https://brickschema.org/schema/1.0.2/building_example#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix tag: <https://brickschema.org/schema/1.1/BrickTag#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bldg:Hot_Water_Supply_Zone a brick:Location .
bldg:Thermal_Zone a brick:Location .
bldg:Living_Room a brick:Location .
bldg:Dining_Room a brick:Location .
bldg:Attic a brick:Location .
bldg:Kitchen a brick:Location .
bldg:Utility a brick:Location .
bldg:Entry_Hallway a brick:Location .
bldg:Master_Bedroom a brick:Location .
bldg:Bedroom_2 a brick:Location .
bldg:Bedroom_3 a brick:Location .
bldg:Bedroom_4 a brick:Location .
bldg:Mudroom a brick:Location .
bldg:Multiple a brick:Location .
bldg:Outdoor a brick:Location .
bldg:Master_Bathroom a brick:Location .
bldg:Bathroom_1 a brick:Location .
bldg:Bathroom_2 a brick:Location .
bldg:First_Floor a brick:Location .
bldg:Second_Floor a brick:Location .
bldg:Basement a brick:Location .
bldg:Lighting_Zone a brick:Location .
bldg:Lighting_Zone brick:hasPart bldg:Attic.
bldg:Lighting_Zone brick:hasPart bldg:Basement.
bldg:Lighting_Zone brick:hasPart bldg:First_Floor.
bldg:Lighting_Zone brick:hasPart bldg:Second_Floor.
bldg:First_Floor brick:hasPart bldg:Living_Room.
bldg:First_Floor brick:hasPart bldg:Entry_Hallway.
bldg:First_Floor brick:hasPart bldg:Bathroom_1.
bldg:First_Floor brick:hasPart bldg:Bedroom_4.
bldg:First_Floor brick:hasPart bldg:Dining_Room.
bldg:First_Floor brick:hasPart bldg:Kitchen.
bldg:First_Floor brick:hasPart bldg:Mudroom.
bldg:First_Floor brick:hasPart bldg:Utility.
bldg:Second_Floor brick:hasPart bldg:Bathroom_2.
bldg:Second_Floor brick:hasPart bldg:Bedroom_2.
bldg:Second_Floor brick:hasPart bldg:Bedroom_3.
bldg:Second_Floor brick:hasPart bldg:Master_Bedroom.
bldg:Second_Floor brick:hasPart bldg:Master_Bethroom.

bldg:AHU01 a brick:AHU ;

"""

In [13]:
def get_tail(room, df, ahu):
    tail = ahu +""" brick:feeds bldg:"""+fix_location_name(room)+""" . \n"""

    for index, row in df.iterrows():
        row_data_label = str(row['Data Label']).strip()
        row_measurement_location = str(row['Measurement Location']).strip()
        if row_measurement_location==room:
            parsed_row_data_label = row_data_label.split("_")
            sub_system = parsed_row_data_label[0]
            data_label = parsed_row_data_label[1]
            tail+=ahu+" brick:hasPoint bldg:"+data_label.replace(" ","")+" .\n"
    return tail[:-2]+"."

def parse():
    body = "\n"
    tail = "\n"
    
    for index, row in df.iterrows():
        row_data_label = str(row['Data Label']).strip()
        row_measurement_location = str(row['Measurement Location']).strip()
        
        #if row_measurement_location=="Living Room" or row_measurement_location=="Basement":
        if True:
            parsed_row_data_label = row_data_label.split("_")
            sub_system = parsed_row_data_label[0]
            data_label = parsed_row_data_label[1]
            body+="bldg:"+fix_location_name(row_measurement_location)+" brick:hasPoint "+"bldg:"+data_label.replace(" ","")+" .\n"
            if "DewpointSupplyAir" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Supply_Air_Dewpoint_Sensor .\n"
            elif "DewpointReturnAir" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Return_Air_Dewpoint_Sensor .\n"
            elif "Air" in data_label and "Temp" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Air_Temperature_Sensor .\n"
            elif "ColdFlow" in data_label or "HotFlow" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Water_Usage_Sensor .\n"
            elif "Flow" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Flow_Sensor .\n"
            elif "flowrate" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Air_Flow_Sensor .\n"
            elif "Airflow" in data_label:
                body+="bldg:"+data_label+ " a brick:Air_Flow_Sensor .\n"
            elif "EnergyTotal" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Energy_Usage_Sensor .\n"
            elif "EnergyPlugs" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Energy_Usage_Sensor .\n"
            elif "Energy" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Energy_Usage_Sensor .\n"
            elif "EnergywithStandby" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Energy_Usage_Sensor .\n"
            elif "PowerwithStandby" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Electrical_Power_Sensor .\n"
            elif "PowerTotal" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Energy_Usage_Sensor .\n"
            elif "Power" in data_label and "Plug" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Power_Sensor .\n"
            elif "Power" in data_label and "Plug" not in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Thermal_Power_Sensor .\n"
            elif "Status" in data_label in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:On_Off_Status .\n"
            elif "RH" in data_label in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Relative_Humidity_Sensor .\n"
            elif "RadiantTemp" in data_label in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Air_Temperature_Sensor .\n"
            elif "RoomTempBasementHPWH" in data_label in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Temperature_Sensor .\n"
            elif "RoomTemp" in data_label in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Zone_Air_Temperature_Sensor .\n"
            elif "WaterTemp" in data_label in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Water_Temperature_Sensor .\n"
            elif "Dewpoint" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Dewpoint_Sensor .\n"
            elif "pressuredifferential" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Differential_Pressure_Sensor .\n"
            elif "Temp" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Temperature_Sensor .\n"
            elif "WaterVolume" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Water_Level_Sensor .\n"
            elif "WindDirection" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Wind_Direction_Sensor .\n"
            elif "WindSpeed" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Wind_Speed_Sensor .\n"
            elif "Amps" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Current_Sensor .\n"
            elif "StringCurrent" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Current_Sensor .\n"
            elif "Frequency" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Frequency_Sensor .\n"
            elif "PVInsolation" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Solar_Radiance_Sensor .\n"
            elif "Frequency" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Frequency_Sensor .\n"
            elif "Voltage" in data_label or "Volts" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Voltage_Sensor .\n"
            elif "Watts" in data_label and "Total" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Power_Sensor .\n"
            elif "WhoursDelivered" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Energy_Usage_Sensor .\n"
            elif "PSPSHW" in data_label:
                body+="bldg:"+data_label.replace(" ","")+ " a brick:Solar_Radiance_Sensor .\n"
            else:
                print(" not found: ", data_label)
    
    ahu = "bldg:AHU01"
    tail = ahu + """ a brick:AHU ;\n"""
    tail += get_tail("Living Room", df, ahu)
    tail += "\n"+get_tail("Basement", df, ahu)
    tail += "\n"+get_tail("Kitchen", df, ahu)
    tail += "\n"+get_tail("Dining Room", df, ahu)
    tail += "\n"+get_tail("Utility", df, ahu)
    tail += "\n"+get_tail("Entry Hallway", df, ahu)
    tail += "\n"+get_tail("MBedroom", df, ahu)
    tail += "\n"+get_tail("Bedroom2", df, ahu)
    tail += "\n"+get_tail("Bedroom3", df, ahu)
    tail += "\n"+get_tail("Bedroom4", df, ahu)
    tail += "\n"+get_tail("Mudroom", df, ahu)
    tail += "\n"+get_tail("Multiple", df, ahu)
    tail += "\n"+get_tail("Outdoor", df, ahu)
    tail += "\n"+get_tail("MBath", df, ahu)
    tail += "\n"+get_tail("Bath1", df, ahu)
    tail += "\n"+get_tail("Bath2", df, ahu)

    return body,tail


In [14]:
body,tail = parse()
print()
print(header+"\n"+body+"\n"+tail)


@prefix bldg: <https://pages.nist.gov/netzero/data.html#data_dictionary#> .
@prefix brick: <https://brickschema.org/schema/1.1/Brick#> . 
@prefix btag: <https://brickschema.org/schema/1.0.2/BrickTag#> .
@prefix gtc: <https://brickschema.org/schema/1.0.2/building_example#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix tag: <https://brickschema.org/schema/1.1/BrickTag#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bldg:Hot_Water_Supply_Zone a brick:Location .
bldg:Thermal_Zone a brick:Location .
bldg:Living_Room a brick:Location .
bldg:Dining_Room a brick:Location .
bldg:Attic a brick:Location .
bldg:Kitchen a brick:Location .
bldg:Utility a brick:Location .
bldg:Entry_Hallway a brick:Location .
bldg:Master_Bedroom a brick:Location .
bldg:Bedroom_2 a brick:Location .
bldg:Bedroom_3 a brick:Location